In [2]:
import numpy as np
from numba import float64,int64,njit,prange,optional

from vect_rcantile import RE, R2D, CE, EPSILON

In [3]:
@njit('float64[:](float64, float64)', parallel=True, fastmath=True)
def truncate_lnglat(lng, lat):
    result = np.zeros((2), dtype=float64)
    if lng > 180.0:
        result[0] = 180.0
    elif lng < -180.0:
        result[0] = -180.0
    else:
        result[0] = lng

    # Truncate latitudes: similarly, directly modifying the output array
    if lat > 90.0:
        result[1] = 90.0
    elif lat < -90.0:
        result[1] = -90.0
    else:
        result[1] = lat
    return result

In [4]:
@njit('float64[:](float64, float64, boolean)', parallel=True, fastmath=True)
def xy(lng, lat, truncate=False):
    result = np.zeros((2), dtype=float64)
    if truncate:
        lng, lat = truncate_lnglat(lng, lat)

    result[0] = RE * np.radians(lng)

    if lat <= -90:
        result[1] = -np.inf
    elif lat >= 90:
        result[1] = np.inf
    else:
        result[1] = RE * np.log(np.tan((np.pi * 0.25) + (0.5 * np.radians(lat))))

    return result

In [5]:
@njit('float64[:](float64, float64, boolean)', parallel=True, fastmath=True)
def lnglat(x, y, truncate=False):
    result = np.zeros((2), dtype=float64)
    result[0], result[1] = (
        x * R2D / RE,
        ((np.pi * 0.5) - 2.0 * np.arctan(np.exp(-y / RE))) * R2D,
    )
    if truncate:
        result[0], result[1] = truncate_lnglat(result[0], result[1])
    return result

In [6]:
@njit('float64[:](int64, int64, int64)', parallel=True, fastmath=True)
def ul(x, y, zoom):
    result = np.zeros((2), dtype=float64)
    Z2 = 2 ** zoom
    lon_deg = x / Z2 * 360.0 - 180.0
    lat_rad = np.arctan(np.sinh(np.pi * (1 - 2 * y / Z2)))
    lat_deg = np.degrees(lat_rad)
    result[0] = lon_deg
    result[1] = lat_deg
    return result

In [7]:
@njit('float64[:](int64,int64,int64)', parallel=True, fastmath=True)
def bounds(x, y, zoom):
    result = np.zeros((4), dtype=float64)
    Z2 = 2 ** zoom
    result[0] = x / Z2 * 360.0 - 180.0
    result[1] = np.degrees(np.arctan(np.sinh(np.pi * (1 - 2 * (y + 1) / Z2))))
    result[2] = (x + 1) / Z2 * 360.0 - 180.0
    result[3] = np.degrees(np.arctan(np.sinh(np.pi * (1 - 2 * y / Z2))))
    return result

In [8]:
@njit('float64[:](int64,int64,int64)', parallel=True, fastmath=True)
def xy_bounds(x, y, zoom):
    result = np.zeros((4), dtype=float64)
    tile_size = CE / 2 ** zoom
    result[0] = x * tile_size - CE / 2
    result[1] = result[0] + tile_size
    result[2] = CE / 2 - y * tile_size
    result[3] = result[2] - tile_size
    return result

In [9]:
@njit('float64[:](float64,float64)', parallel=True, fastmath=True)
def _xy(lng, lat):
    result = np.zeros((2), dtype=float64)
    result[0] = lng / 360.0 + 0.5
    result[1] = 0.5 - 0.25 * np.log((1.0 + np.sin(np.radians(lat))) / (1.0 - np.sin(np.radians(lat)))) / np.pi
    return result

In [10]:
@njit('int64[:](float64,float64,int64)', parallel=True, fastmath=True)
def tile(lng, lat, zoom):
    result = np.zeros((3), dtype=int64)
    x = lng / 360.0 + 0.5
    y = 0.5 - 0.25 * np.log((1.0 + np.sin(np.radians(lat))) / (1.0 - np.sin(np.radians(lat)))) / np.pi
    Z2 = 2 ** zoom
    result[2] = zoom
    if x <= 0:
        result[0] = 0
    elif x >= 1:
        result[0] = np.int64(Z2 - 1)
    else:
        result[0] = np.int64(np.floor((x + EPSILON) * Z2))

    if y <= 0:
        result[1] = 0
    elif y >= 1:
        result[1] = np.int64(Z2 - 1)
    else:
        result[1] = np.int64(np.floor((y + EPSILON) * Z2))
    return result

In [11]:
print(tile(55.679134, 37.263656,10))

[670 397  10]


In [12]:
@njit('int64[:](int64,int64,int64)', parallel=True, fastmath=True)
def quadkey(xtile,ytile,zoom):
    """Get the quadkey of a tile

    Parameters
    ----------
    tile : Tile or sequence of int
        May be be either an instance of Tile or 3 ints, X, Y, Z.

    Returns
    -------
    str

    """
    result = np.zeros((zoom), dtype=int64)
    for i in prange(zoom):
        digit = 0
        bit = zoom - i - 1
        if (xtile >> bit) & 1:
            digit += 1
        if (ytile >> bit) & 1:
            digit += 2
        result[i] = digit
    return result

In [13]:
@njit('int64[:](int64[:])', parallel=True, fastmath=True)
def quadkey_to_tile(quadkey):
    x = y = 0
    z = len(quadkey)
    result = np.array([x,y,z], dtype=np.int64)
    for i in prange(z):
        bit = z - i - 1
        mask = 1 << bit
        if quadkey[i] == 1:
            result[0] |= mask
        elif quadkey[i] == 2:
            result[1] |= mask
        elif quadkey[i] == 3:
            result[0] |= mask
            result[1] |= mask
        # No need to check for '0' as it does not affect x or y
    return result

In [14]:
print(quadkey_to_tile(quadkey(670,397,10)))

[670 397  10]


In [15]:
from vect_rcantile.constants import LL_EPSILON


@njit('int64[:,:](float64,float64,float64,float64,int64,boolean)', parallel=True, fastmath=True)
def generate_tiles(w, s, e, n, zoom, truncate=False):
    if truncate:
        w, s = truncate_lnglat(w, s)
        e, n = truncate_lnglat(e, n)

    ul_tile = tile(w, n, zoom)
    lr_tile = tile(e - LL_EPSILON, s + LL_EPSILON, zoom)

    num_tiles = (lr_tile[0] - ul_tile[0] + 1) * (lr_tile[1] - ul_tile[1] + 1)

    result = np.zeros((num_tiles, 3), dtype=np.int64)
    index = 0
    while index < num_tiles:
        result[index][0] = index % (lr_tile[0] - ul_tile[0] + 1) + ul_tile[0]
        result[index][1] = index // (lr_tile[0] - ul_tile[0] + 1) + ul_tile[1]
        result[index][2] = zoom
        index += 1
    return result

In [16]:
for tile in generate_tiles(34.347655,53.917331,44.367186,58.041185, 10, False):
    print(tile)

[609 308  10]
[610 308  10]
[611 308  10]
[612 308  10]
[613 308  10]
[614 308  10]
[615 308  10]
[616 308  10]
[617 308  10]
[618 308  10]
[619 308  10]
[620 308  10]
[621 308  10]
[622 308  10]
[623 308  10]
[624 308  10]
[625 308  10]
[626 308  10]
[627 308  10]
[628 308  10]
[629 308  10]
[630 308  10]
[631 308  10]
[632 308  10]
[633 308  10]
[634 308  10]
[635 308  10]
[636 308  10]
[637 308  10]
[638 308  10]
[609 309  10]
[610 309  10]
[611 309  10]
[612 309  10]
[613 309  10]
[614 309  10]
[615 309  10]
[616 309  10]
[617 309  10]
[618 309  10]
[619 309  10]
[620 309  10]
[621 309  10]
[622 309  10]
[623 309  10]
[624 309  10]
[625 309  10]
[626 309  10]
[627 309  10]
[628 309  10]
[629 309  10]
[630 309  10]
[631 309  10]
[632 309  10]
[633 309  10]
[634 309  10]
[635 309  10]
[636 309  10]
[637 309  10]
[638 309  10]
[609 310  10]
[610 310  10]
[611 310  10]
[612 310  10]
[613 310  10]
[614 310  10]
[615 310  10]
[616 310  10]
[617 310  10]
[618 310  10]
[619 310  10]
[620 3